# PyCaret Regression — Kaggle
For Kaggle: use **Add data** to attach a dataset. CSV files appear under `/kaggle/input/...`.
Run the first cell to install PyCaret 3.3.2.


Assignment Done by :- **Dev Mulchandani**

In [1]:
%pip -q install -U pip
%pip -q install 'pycaret==3.3.2'
import sys
print('Python:', sys.version)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.3 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.40.3 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.7, but you have notebook 6.5.4 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.1.4 which is incompatible.
google-colab 

In [2]:
# List CSVs under /kaggle/input and pick one by index
import glob, pandas as pd

csvs = sorted(glob.glob('/kaggle/input/**/*.csv', recursive=True))
if not csvs:
    raise SystemExit("No CSVs found. Click 'Add Data' in Kaggle and attach your dataset, then rerun.")

for i, p in enumerate(csvs):
    print(f"{i}: {p}")

idx = int(input("Enter the index of the CSV to load: "))
DATA_PATH = csvs[idx]
print("✅ Using:", DATA_PATH)

data = pd.read_csv(DATA_PATH)
print("Shape:", data.shape)
display(data.head())


0: /kaggle/input/test-regression/test.csv
1: /kaggle/input/train-regression/train.csv


Enter the index of the CSV to load:  1


✅ Using: /kaggle/input/train-regression/train.csv
Shape: (1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
# --- Run this cell FIRST (before importing pycaret.*) ---
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""   # hide GPUs so RAPIDS/GPU paths are skipped
# If RAPIDS bits are present, uninstall so PyCaret won't touch them
!pip -q uninstall -y cuml cudf cuml-cu12 cudf-cu12 cupy-cuda12x cupy-cuda11x cupy || true


In [4]:
from pycaret.regression import *
TARGET = input('Enter target column name: ')
exp = setup(data, target=TARGET, session_id=123, use_gpu=False)
best = compare_models()
evaluate_model(best)
save_model(best, 'best_regression_model')

Enter target column name:  SalePrice


,Description,Value
0,Session id,123
1,Target,SalePrice
2,Target type,Regression
3,Original data shape,"(1460, 81)"
4,Transformed data shape,"(1460, 279)"
5,Transformed train set shape,"(1021, 279)"
6,Transformed test set shape,"(439, 279)"
7,Numeric features,37
8,Categorical features,43
9,Rows with missing values,100.0%


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,15387.2909,792470919.4914,27171.5462,0.8847,0.1287,0.0892,5.2750
gbr,Gradient Boosting Regressor,17276.3461,847357928.1180,28313.9245,0.8723,0.1377,0.1000,0.6190
lightgbm,Light Gradient Boosting Machine,17701.1514,1019922835.2337,31002.5633,0.8484,0.1449,0.1020,24.8790
xgboost,Extreme Gradient Boosting,19068.1370,1120869534.7547,32187.5030,0.8345,0.1530,0.1102,0.5500
rf,Random Forest Regressor,19042.9144,1117550843.2233,32426.1601,0.8345,0.1545,0.1114,1.3650
et,Extra Trees Regressor,18873.1767,1163678875.5551,32786.9480,0.8327,0.1513,0.1095,1.3780
llar,Lasso Least Angle Regression,18747.1670,1324909515.6131,33700.0973,0.8028,0.1707,0.1136,0.3120
ada,AdaBoost Regressor,25963.6459,1417400709.1888,37051.1993,0.7868,0.2056,0.1679,0.4820
ridge,Ridge Regression,20081.3998,1526032661.7945,36561.0633,0.7668,0.2087,0.1213,0.2870
en,Elastic Net,21115.3002,1782962549.6151,38958.8760,0.7382,0.1739,0.1218,0.4170


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['Id', 'MSSubClass', 'LotFrontage',
                                              'LotArea', 'OverallQual',
                                              'OverallCond', 'YearBuilt',
                                              'YearRemodAdd', 'MasVnrArea',
                                              'BsmtFinSF1', 'BsmtFinSF2',
                                              'BsmtUnfSF', 'TotalBsmtSF',
                                              '1stFlrSF', '2ndFlrSF',
                                              'LowQualFinSF', 'GrLivArea',
                                              'BsmtFullBath', 'BsmtHalfBath',
                                              'FullBath', 'Hal...
                                                                     'Exterior2nd',
                                                                     'MasVnrType',
              

In [5]:
# ---- Robust prediction + submission for House Prices ----
from pycaret.regression import load_model, predict_model
import pandas as pd, glob, re

# 1) Load saved model
model = load_model("best_regression_model")

# 2) Pick your TEST csv under /kaggle/input (you already picked index 0 earlier; reuse if you like)
csvs = sorted(glob.glob('/kaggle/input/**/*.csv', recursive=True))
for i, p in enumerate(csvs):
    print(f"{i}: {p}")
try:
    idx = int(input("\nEnter the index of the TEST CSV (the file WITHOUT SalePrice): "))
except Exception:
    idx = 0
TEST_PATH = csvs[idx]
print("✅ Using:", TEST_PATH)

# 3) Read test and run predictions
test = pd.read_csv(TEST_PATH)
preds = predict_model(model, data=test)   # adds a prediction column

# 4) Figure out the prediction column name returned by PyCaret
pred_col = None
for c in preds.columns:
    if c.lower() in ("label", "prediction_label", "prediction"):
        pred_col = c
        break
if pred_col is None:
    raise KeyError(
        f"Could not find prediction column in preds. Available columns: {list(preds.columns)}"
    )

# 5) Choose / create an Id column for submission
id_col = None
# exact 'Id' first
if "Id" in test.columns:
    id_col = "Id"
else:
    # look for something that looks like an id
    candidates = [c for c in test.columns if re.search(r"\bid\b", c, flags=re.I)]
    if candidates:
        id_col = candidates[0]
        # also copy it to 'Id' for Kaggle House Prices format
        test["Id"] = test[id_col]
        id_col = "Id"
    else:
        # last resort: make a simple Id 1..N
        test["Id"] = range(1, len(test) + 1)
        id_col = "Id"

# 6) Build submission in the official format (Id + SalePrice)
submission = pd.DataFrame({
    "Id": test[id_col],
    "SalePrice": preds[pred_col]
})

submission.to_csv("submission.csv", index=False)
print("✅ submission.csv saved in /kaggle/working")
submission.head()


Transformation Pipeline and Model Successfully Loaded
0: /kaggle/input/test-regression/test.csv
1: /kaggle/input/train-regression/train.csv



Enter the index of the TEST CSV (the file WITHOUT SalePrice):  0


✅ Using: /kaggle/input/test-regression/test.csv


✅ submission.csv saved in /kaggle/working


,Id,SalePrice
0,1461,119623.800204
1,1462,164549.436316
2,1463,183728.263350
3,1464,190115.144086
4,1465,187445.930368
